# Web Robots  

> Web robots that automates the process of obtaining full text papers (and other interactions with the web)


In [1]:
#| default_exp utils.web_robot

In [5]:
#| export

import pickle as pkl
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from splinter import Browser
from selenium.webdriver.chrome.service import Service
import re
from random import randint
from time import sleep
import requests
import os

In [10]:
#| export

def retrieve_pdf_from_doidotorg(doi, base_dir):
    browser = Browser()#, headless=True)
    doi = doi.replace('https://doi.org/', '').replace('doi', '')
    stem = doi.split('/')[0]
    if os.path.exists(base_dir+'/'+stem) is False:
        os.makedirs(base_dir+'/'+stem)
    hrefs = set()
    try:
        # visit bioRxiv's paper page
        browser.visit('https://doi.org/'+doi)
        sleep(randint(5,10)*0.1)
        for link in browser.find_by_css('a'):
            if link['href'] is not None and 'pdf' in link['href']:
                hrefs.add(link['href'])
                        
    except Exception as e:
        print(e)

    browser.quit()

    link = min(hrefs, key=len) # prints "i"
    response = requests.get(link)
    with open(base_dir+'/'+doi+'.pdf', 'wb') as f:
        f.write(response.content)

#retrieve_pdf_from_doidotorg('10.1083/jcb.202204093', '/Users/gburns/alhazen/em_tech/temp/')

In [4]:
#| export

def retrieve_full_text_links_from_biorxiv(doi, base_dir):
    browser = Browser()#, headless=True)
    doi = doi.replace('https://doi.org/', '').replace('doi', '')
    if doi.startswith('10.1101/') is False:
        print('Not a BioRxiv DOI')
        return None
    if os.path.exists(base_dir+'/10.1101') is False:
        os.makedirs(base_dir+'/10.1101')
    hrefs = []
    try:
        # visit bioRxiv's paper page
        browser.visit('https://www.biorxiv.org/content/'+doi)
        sleep(randint(5,10)*0.1)
        for link in browser.find_by_css('a[class="dropdown-link"]'):
            if link['href']:
                hrefs.append(link['href'])
                        
    except Exception as e:
        print(e)

    browser.quit()

    for link in hrefs:
        if link.endswith('.pdf'):
            response = requests.get(link)
        
            with open(base_dir+'/'+doi+'.pdf', 'wb') as f:
                f.write(response.content)
        elif link.endswith('.xml'):
            response = requests.get(link)
            with open(base_dir+'/'+doi+'.nxml', 'wb') as f:
                f.write(response.content)
        

In [1]:
#| export

def execute_search_on_biorxiv(search_term):
    browser = Browser()#, headless=True)
    all_dois = []

    try:
        # visit bioRxiv's search page
        browser.visit('https://www.biorxiv.org/search')

        # fill in the search form
        sleep(randint(5,10)*0.1)
        print('fill search term')
        browser.find_by_id('edit-txtsimple').fill(search_term)

        sleep(randint(5,10)*0.1)
        print('scroll to bottom')
        #browser.scroll_to('bottom')
        
        print('click search button')        
        browser.find_by_css('a[class="search-choice-close"]').click()    
        #browser.find_by_css('input[class="form-submit"]').click()    
        browser.find_by_id('edit-actions').find_by_value('Search').click()
        
        sleep(randint(5,10)*0.1)
        print('load next page')
        # Extract the number of results
        formatted_string = browser.find_by_id('page-title').text
        
        # Use regular expressions to get the number from a string formatted 'XXX Results'
        m = re.search(r'\d+ Results', formatted_string)
        if m:
            num_results = int(re.search(r'\d+', formatted_string).group())
            
            loop_count = 0
            while True:
                
                # Extract each result from the list of the web page
                doi_links = browser.find_by_css('span[class="highwire-cite-metadata-doi highwire-cite-metadata"]')
                all_dois.extend([re.sub('doi: ', '', t.text) for t in doi_links])
                #
                # Is next button absent?
                next_button_not_present = browser.is_element_not_present_by_css('a[class="link-icon link-icon-after"]')

                if next_button_not_present:
                    break
            
                # Find the next button on the page
                next_button = browser.find_by_css('a[class="link-icon link-icon-after"]')

                # Click the next button
                next_button.click()
                sleep(randint(5,10)*0.1)
                #print('load page number'+str(loop_count))

                loop_count += 1
                if loop_count > 100:
                    break
            
    except Exception as e:
        print(e)

    browser.quit()
    return all_dois

#all_dois = run_biorxiv_assistant('cellxgene')
#print(all_dois)

fill search term
scroll to bottom
click search button
load next page
['https://doi.org/10.1101/2020.08.28.270652', 'https://doi.org/10.1101/2021.04.05.438318', 'https://doi.org/10.1101/2023.07.11.548639', 'https://doi.org/10.1101/2023.10.07.561331', 'https://doi.org/10.1101/2023.09.10.557072', 'https://doi.org/10.1101/2023.09.27.559702', 'https://doi.org/10.1101/2021.09.30.462602', 'https://doi.org/10.1101/2023.01.03.522578', 'https://doi.org/10.1101/2023.10.16.562533', 'https://doi.org/10.1101/2023.06.16.544954', 'https://doi.org/10.1101/2023.09.21.558902', 'https://doi.org/10.1101/2023.06.08.542671', 'https://doi.org/10.1101/2023.08.18.553912', 'https://doi.org/10.1101/2023.09.28.23295408', 'https://doi.org/10.1101/2023.01.21.514916', 'https://doi.org/10.1101/2023.07.18.549537', 'https://doi.org/10.1101/2023.06.21.545951', 'https://doi.org/10.1101/2023.08.23.554343', 'https://doi.org/10.1101/2023.08.17.553606', 'https://doi.org/10.1101/2023.05.09.540014', 'https://doi.org/10.1101/202